# GraphSAGE with PPI Dataset

A GraphSAGE implementation from the paper [Inductive Representation Learning on Large Graphs William L. Hamilton et al.](https://arxiv.org/abs/1706.02216)

###installing spektral

In [1]:
!pip install --quiet spektral

##importing dependencies

In [2]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.random import set_seed

import spektral
from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GATConv , GraphSageConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess

set_seed(0)

## Load Data

The datasets used in the paper

Inductive Representation Learning on Large Graphs
William L. Hamilton et al.

The PPI dataset (originally Stark et al. (2006)) for inductive node classification uses positional gene sets, motif gene sets and immunological signatures as features and gene ontology sets as labels.

In [3]:
dataset = spektral.datasets.graphsage.GraphSage("PPI")

### Defining masks

In [4]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

### Defining Parameters

In [5]:
# Parameters
channels = 8  # Number of channels in each head of the first GAT layer
n_attn_heads = 8  # Number of attention heads in first GAT layer
dropout = 0.6  # Dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # L2 regularization rate
learning_rate = 5e-3  # Learning rate
epochs = 200  # Number of training epochs
patience = 100  # Patience for early stopping

N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes

## Defining the Model

Implementation with "mean" as aggregator.

This layer computes:

$$
X^{'} = [AGGREGATE(X)||X]W+b
$$

$$
X^{'} = \frac{X^{'}}{||X||}
$$
where AGGREGATE is a function to aggregate a node's neighbourhood. The supported aggregation methods are: sum, mean, max, min, and product.



```
spektral.layers.GraphSageConv(channels,
                              aggregate='mean',
                              activation=None,
                              use_bias=True,
                              kernel_initializer='glorot_uniform',
                              bias_initializer='zeros',
                              kernel_regularizer=None,
                              bias_regularizer=None,
                              activity_regularizer=None,
                              kernel_constraint=None,
                              bias_constraint=None)
```



In [6]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True)
do_1 = Dropout(dropout)(x_in)
gc_1 = GraphSageConv(channels,
              aggregate='mean',
              activation="elu",
              use_bias=True,
              kernel_initializer='glorot_uniform',
              bias_initializer='zeros',
              kernel_regularizer=l2(l2_reg),
              bias_regularizer=l2(l2_reg),
              )([do_1, a_in])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GraphSageConv(
              n_out,
              aggregate='mean',
              activation="softmax",
              use_bias=True,
              kernel_initializer='glorot_uniform',
              bias_initializer='zeros',
              kernel_regularizer=l2(l2_reg),
              bias_regularizer=l2(l2_reg)
               )([do_2, a_in])

### Compiling and Building the Model

In [7]:
# Build model
model = Model(inputs=[x_in, a_in], outputs=gc_2)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 50)           0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 56944)]      0                                            
__________________________________________________________________________________________________
graph_sage_conv (GraphSageConv) (None, 8)            808         dropout[0][0]                    
                                                                 input_2[0][0]                

## Training

In [8]:
# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 178.6083 - acc: 0.0069 - val_loss: 171.1186 - val_acc: 0.0020
Epoch 2/200
1/1 [==============================] - 1s 800ms/step - loss: 178.5860 - acc: 0.0039 - val_loss: 171.1008 - val_acc: 0.0015
Epoch 3/200
1/1 [==============================] - 1s 811ms/step - loss: 178.5728 - acc: 0.0034 - val_loss: 171.0828 - val_acc: 0.0012
Epoch 4/200
1/1 [==============================] - 1s 827ms/step - loss: 178.5615 - acc: 0.0032 - val_loss: 171.0643 - val_acc: 0.0011
Epoch 5/200
1/1 [==============================] - 1s 801ms/step - loss: 178.5507 - acc: 0.0027 - val_loss: 171.0443 - val_acc: 7.6758e-04
Epoch 6/200
1/1 [==============================] - 1s 799ms/step - loss: 178.5305 - acc: 0.0032 - val_loss: 171.0226 - val_acc: 0.0011
Epoch 7/200
1/1 [==============================] - 1s 793ms/step - loss: 178.5181 - acc: 0.0025 - val_loss: 170.9991 - val_acc: 7.6758e-04
Epoch 8/200
1/1 [==============================] -

## Evaluation

In [9]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 359ms/step - loss: 172.8377 - acc: 0.0020
Done.
Test loss: 172.83767700195312
Test accuracy: 0.001991310389712453
